In [20]:
# XGBoost data prep
#remove white space in columns
#NOTE: GENDER_bin=1 is female and GENDER_bin=0 is males

df = pd.read_csv("df_sum_score_py.csv")
df.replace(' ','_',regex=True,inplace=True)

# there is 14 0's in STRATUM and since this is only approximately 0.116% of the entire data, we do not need to do anything. 
# XGBoost is very well at handling missing data, and we just need to ensure that it's set to 0. 
df.STRATUM.isin(['0']).sum()

,Unnamed: 0,GENDER,STRATUM,CR_S11,CC_S11,ENG_S11,CR_PRO,CC_PRO,ENG_PRO,COL_GRADE_AVG,HI_GRADE_AVG,GENDER_bin
0,0,F,Stratum_4,81,61,82,93,71,93,85.666667,74.666667,1
1,1,F,Stratum_5,75,66,88,38,86,98,74.000000,76.333333,1
2,2,M,Stratum_2,49,38,42,1,18,43,20.666667,43.000000,0
3,3,F,Stratum_2,55,51,73,35,76,80,63.666667,59.666667,1
4,4,M,Stratum_4,65,76,92,94,98,100,97.333333,77.666667,0


In [26]:
# Splitting the data 
#We want to predict the score of the students in college. 

X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_S11','CC_S11','ENG_S11','CR_PRO','CC_PRO','ENG_PRO'], axis=1).copy()
y=df['COL_GRADE_AVG'].copy()

# X is the data which will be used to make predictions, and y contains the data we want to predict. 
# Now we will continue to formatting X to make it suitable for making a model with XGBoost. 

In [40]:
# One-hot encoding 
X.dtypes
# The object columns we need to inspect to ensure that they are what we need them to be and after that we will do one-hot encoding. 
# One hot encoding is used to make the categorical varoiable STRATUM work in the model. 
# What is gonna happen is that the categorical variable is becoming multiple columns of binary values. 
# One hot encoding works great for trees and this is the motivation for using this method. 
X_encoded = pd.get_dummies(X,columns=['STRATUM']).head()

In [ ]:
# NOw the problem is, that our y is not 1's and 0's...

In [9]:
import numpy as np
import pandas as pd
# Loading the data
df = pd.read_csv("df_sum_score_py.csv")
# C:\Dokumenter\Dokumenter\UNI\7.semester\Bachelor\Bachelor_Gabi_Kat\src\Algoritmer\df_sum_score_py.csv

# Creating the data
# features
df_document_features = df.copy()
df_document_features = df_document_features[["GENDER_bin", "HI_GRADE_AVG"]]
df_document_features = df_document_features.values.tolist()
doc_features = np.array(df_document_features)
#scores
doc_scores = df["COL_GRADE_AVG"].values

query = df.index.values
df.index

RangeIndex(start=0, stop=12411, step=1)

In [10]:
from sklearn.model_selection import GroupShuffleSplit


gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(df, groups=df.index.values)


In [ ]:

X_train_inds, X_test_inds = next(gss)

train_data= df.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['id','rank'])]
y_train = train_data.loc[:, train_data.columns.isin(['rank'])]

groups = train_data.groupby('id').size().to_frame('size')['size'].to_numpy()

test_data= df.iloc[X_test_inds]

#We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['rank'])]
y_test = test_data.loc[:, test_data.columns.isin(['rank'])]

Umiddelbart det som skal ske her er at vi skal bare fordele data i træning og test. Dette tror jeg hellere at jeg vil gøre selv i stedet for hans kode. 

In [3]:

import xgboost as xgb

model = xgb.XGBRanker(  
    tree_method='gpu_hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )

model.fit(X_train, y_train, group=groups, verbose=True)

ModuleNotFoundError: No module named 'xgboost'

In [4]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['id'])])
  
predictions = (data.groupby('id')
               .apply(lambda x: predict(model, x)))

NameError: name 'data' is not defined